In [1]:
import torch
print(torch.__version__)

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

torch.cuda.empty_cache()



2.0.1+cu117
GPU is available


In [42]:
import torch
import torch.optim as optim
from torchvision import datasets, models, transforms

# Load the pre-trained model (e.g., ResNet)
model = models.resnet152(pretrained=True)

# Freeze all the layers in the model
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer of the model to match the number of classes in the new dataset
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)  # num_classes should be the number of classes in your new dataset

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

train_dir = "images"  # replace train_dir with the path to your data
# Define the data augmentation and normalization for training
# Add more transforms for data augmentation
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization coefficients for ImageNet
])

train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)  # reduce batch_size


# Training the model
num_epochs = 20
train_loss_history = []
for epoch in range(num_epochs):  # num_epochs should be the number of epochs you want to train for
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, running_loss/len(train_loader)))




c:\Users\leo_b\anaconda3\envs\pmba_new\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\leo_b\anaconda3\envs\pmba_new\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\leo_b/.cache\torch\hub\checkpoints\resnet152-394f9c45.pth
100.0%


Epoch [1/20], Loss: 0.7282
Epoch [2/20], Loss: 0.7530
Epoch [3/20], Loss: 0.6386
Epoch [4/20], Loss: 0.6226
Epoch [5/20], Loss: 0.5777
Epoch [6/20], Loss: 0.5466
Epoch [7/20], Loss: 0.5142
Epoch [8/20], Loss: 0.5289
Epoch [9/20], Loss: 0.5116
Epoch [10/20], Loss: 0.5368
Epoch [11/20], Loss: 0.4573
Epoch [12/20], Loss: 0.4341
Epoch [13/20], Loss: 0.3988
Epoch [14/20], Loss: 0.4004
Epoch [15/20], Loss: 0.4259
Epoch [16/20], Loss: 0.3921
Epoch [17/20], Loss: 0.4048
Epoch [18/20], Loss: 0.4260
Epoch [19/20], Loss: 0.4126
Epoch [20/20], Loss: 0.4539


In [62]:
# Save the state of the model
torch.save(model.state_dict(), 'models/resnet152_aug.pth')

# Load the state of the model
#model.load_state_dict(torch.load('model_weights.pth'))

In [63]:
from torchvision import transforms

# Define the transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to the same size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

# Load the testing data
test_dir = "test_dataset"  # replace test_dir with the path to your testing data
test_data = datasets.ImageFolder(test_dir, transform=transform)  # apply the defined transform


test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

# Switch the model to evaluation mode
model.eval()

# Create a list to store predictions
predictions = []

# Test the model
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Append the predictions to the list
        predictions.extend(predicted.tolist())

# Calculate and print the accuracy
accuracy = float(correct / total)
print('Test Accuracy: %.2f %%' % (100 * accuracy))

# Print the class predictions
print('Class predictions:', predictions)

# Create a reverse mapping from index to class name
idx_to_class = {v: k for k, v in test_data.class_to_idx.items()}

# Transform the predictions list
class_predictions = [idx_to_class.get(idx, 'Unknown') for idx in predictions]

# Print the class predictions
print('Class predictions:', class_predictions)
# Print the ground-truth labels for each image testing data
print('Ground-truth labels:', [idx_to_class.get(idx) for idx in test_data.targets])

Test Accuracy: 75.00 %
Class predictions: [0, 0, 0, 1, 0, 1, 0, 0]
Class predictions: ['correct_seq', 'correct_seq', 'correct_seq', 'wrong_seq', 'correct_seq', 'wrong_seq', 'correct_seq', 'correct_seq']
Ground-truth labels: ['correct_seq', 'correct_seq', 'correct_seq', 'correct_seq', 'wrong_seq', 'wrong_seq', 'wrong_seq', 'wrong_seq']
